In [1]:
!pip install adapters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 8.8 MB/s eta 0:00:00


In [2]:
!pip install transformers[torch]

In [3]:
!pip install datasets

In [4]:

import os
import sys
import json
import time
from datetime import datetime
import torch
from tokenizers import ByteLevelBPETokenizer
from transformers import RobertaConfig
from transformers import RobertaTokenizer
from transformers import RobertaForMaskedLM
from transformers import BertTokenizer
from adapters import BertAdapterModel
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import TrainerCallback
from adapters import AutoAdapterModel
import adapters
from datasets import Dataset
import pandas as pd

NLU_FILE = "/kaggle/input/babylm-prompt-4-nlu/combined.xlsx"

# Load pre-trained BERT tokenizer from Hugging Face
print("Loading tokenizer...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained BERT model from Hugging Face
print("Loading model...")
model = BertAdapterModel.from_pretrained('bert-base-uncased')
adapters.init(model)

# Define the adapter arguments
adapterArgs = adapters.training.AdapterArguments(
  train_adapter=True,
)

# Setup the adapter training (this will add the adapter to the model)
print("Setting up adapter training...")
adapters.training.setup_adapter_training(
  model=model,
  adapter_args=adapterArgs,
  adapter_name='DutchAdapter',
)

model.set_active_adapters('DutchAdapter')

# Define the training arguments. Default training arguments from the adapter library are used
trainingArgs = TrainingArguments(
  learning_rate=1e-4,
  num_train_epochs=2,
  output_dir='DutchAdapter',
  evaluation_strategy="epoch",
)

data = pd.read_excel(NLU_FILE)

dataset = Dataset.from_pandas(data)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['0'], padding="max_length", truncation=True)

# make splits for dataset with 90 % train and rest for test
dataset = dataset.train_test_split(test_size=0.1)

tokenizedDataset = dataset.map(tokenize_function, batched=True)

dataCollator = DataCollatorForLanguageModeling(
  tokenizer=tokenizer, mlm=True, mlm_probability= 0.15
)

# setup the adapter trainer
print("Setting up trainer...")
trainer = adapters.AdapterTrainer(
        model=model,
        args=trainingArgs,
        train_dataset=tokenizedDataset['train'],
        eval_dataset=tokenizedDataset['test'],
        tokenizer=tokenizer,
        data_collator=dataCollator,
    )

# Train the adapter
print("Training adapter...")
trainer.train()

# Save the adapter to the Adapter directory
print("Saving adapter...")
model.save_adapter(adapter_name='DutchAdapter', save_directory='/kaggle/Output/Adapter')

print("Done!")

2024-04-30 08:27:38.687464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 08:27:38.687578: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 08:27:38.816135: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.398200,1.242658
2,1.258500,1.130205


Saving adapter...
Done!
